# Import Data

In [103]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

# To split the dataset into train and test datasets
from sklearn.cross_validation import train_test_split

# To model the Gaussian Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB

# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score
import re

In [104]:
players_2010 = pd.read_csv('Players_2010.csv')

In [105]:
#Import events
events_2010 = pd.read_csv('Events_2010.csv')
events_2011 = pd.read_csv('Events_2011.csv')
events_2012 = pd.read_csv('Events_2012.csv')
events_2013 = pd.read_csv('Events_2013.csv')
events_2014 = pd.read_csv('Events_2014.csv')
events_2015 = pd.read_csv('Events_2015.csv')
events_2016 = pd.read_csv('Events_2016.csv')
events_2017 = pd.read_csv('Events_2017.csv')

#Import players
players_2010 = pd.read_csv('Players_2010.csv')
players_2011 = pd.read_csv('Players_2011.csv')
players_2012 = pd.read_csv('Players_2012.csv')
players_2013 = pd.read_csv('Players_2013.csv')
players_2014 = pd.read_csv('Players_2014.csv')
players_2015 = pd.read_csv('Players_2015.csv')
players_2016 = pd.read_csv('Players_2016.csv')
players_2017 = pd.read_csv('Players_2017.csv')

#Import Massey
massey = pd.read_csv('MasseyOrdinals.csv')

#Import Other files
cities = pd.read_csv('Cities.csv')
conferences = pd.read_csv('Conferences.csv')
conf_tourney = pd.read_csv('ConferenceTourneyGames.csv')
game_cities = pd.read_csv('GameCities.csv')
tourney_results = pd.read_csv('NCAATourneyCompactResults.csv')
detailed_tourney_results = pd.read_csv('NCAATourneyDetailedResults.csv')
tourney_all = pd.read_csv('NCAATourneySeedRoundSlots.csv')
tourney_seeds = pd.read_csv('NCAATourneySeeds.csv')
tourney_slots = pd.read_csv('NCAATourneySlots.csv')
reg_season = pd.read_csv('RegularSeasonCompactResults.csv')
detailed_reg_season = pd.read_csv('RegularSeasonDetailedResults.csv')
seasons = pd.read_csv('Seasons.csv')
second_tourney = pd.read_csv('SecondaryTourneyCompactResults.csv')
detailed_second_tourney = pd.read_csv('SecondaryTourneyTeams.csv')
team_coaches = pd.read_csv('TeamCoaches.csv')
team_conferences = pd.read_csv('TeamConferences.csv')
teams = pd.read_csv('teams.csv')

In [106]:
reg_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


# Preliminary Analysis

### Modify schedule table

In [107]:
#Add the differential
reg_season['Diff'] = reg_season['WScore'] - reg_season['LScore'] 
reg_season['True_Outcome'] = np.where(reg_season['Diff'] > 5, 'Y','N')

#Example query
reg_season[(reg_season['Season'] == 2017) & (reg_season['True_Outcome'] == 'N')].head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Diff,True_Outcome
145291,2017,11,1112,65,1277,63,N,0,2,N
145311,2017,11,1182,65,1259,60,H,0,5,N
145314,2017,11,1188,69,1218,68,A,0,1,N
145318,2017,11,1201,69,1427,66,H,0,3,N
145319,2017,11,1203,75,1271,71,H,0,4,N


In [108]:
massey.head(5)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


### Ratings

In [109]:
#What are the unique rankings? -- Looks like every team gets rated 1-351
massey['SystemName'].sort_values().unique()

array(['7OT', 'ACU', 'ADE', 'AP', 'ARG', 'AUS', 'BBT', 'BCM', 'BD', 'BIH',
       'BKM', 'BLS', 'BNM', 'BOB', 'BOW', 'BP5', 'BPI', 'BRZ', 'BUR',
       'BWE', 'CJB', 'CMV', 'CNG', 'COL', 'CPA', 'CPR', 'CRO', 'CRW',
       'CTL', 'D1A', 'DAV', 'DC', 'DC2', 'DCI', 'DDB', 'DES', 'DII',
       'DOK', 'DOL', 'DUN', 'DWH', 'EBB', 'EBP', 'ECK', 'ENT', 'ERD',
       'ESR', 'FAS', 'FMG', 'FSH', 'GC', 'GRN', 'GRS', 'HAS', 'HAT',
       'HER', 'HKB', 'HKS', 'HOL', 'HRN', 'IMS', 'INP', 'ISR', 'JCI',
       'JEN', 'JNG', 'JON', 'JRT', 'KBM', 'KEL', 'KLK', 'KMV', 'KOS',
       'KPI', 'KPK', 'KRA', 'LMC', 'LOG', 'LYD', 'LYN', 'MAS', 'MB',
       'MCL', 'MGY', 'MIC', 'MKV', 'MOR', 'MPI', 'MSX', 'MUZ', 'MvG',
       'NOL', 'NOR', 'OCT', 'OMY', 'PEQ', 'PGH', 'PH', 'PIG', 'PKL',
       'PMC', 'POM', 'PPR', 'PRR', 'PTS', 'RAG', 'REI', 'REN', 'REW',
       'RIS', 'RM', 'ROG', 'ROH', 'RPI', 'RSE', 'RSL', 'RT', 'RTB', 'RTH',
       'RTP', 'RTR', 'SAG', 'SAP', 'SAU', 'SCR', 'SE', 'SEL', 'SFX',
       'SGR', '

In [110]:
""" Create a table of average of all end of year ratings """

#Drop duplicates from each rating system keep only max rating day
ratings = massey.sort_values('RankingDayNum', ascending = False).drop_duplicates(['Season','SystemName','TeamID'])

#testing - example of how to query on it 
ratings[(ratings['Season'] == 2017) & (ratings['SystemName'] == 'POM')].sort_values('OrdinalRank').head(5)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
3190803,2017,133,POM,1211,1
3191024,2017,133,POM,1437,2
3190903,2017,133,POM,1314,3
3190837,2017,133,POM,1246,4
3191037,2017,133,POM,1452,5


In [111]:
#Get the mean rating for each season team combo across all rating systems
ratings2 = ratings.groupby(['Season', 'TeamID'], as_index = False)['OrdinalRank'].mean()

ratings2.head(2)

,Season,TeamID,OrdinalRank
0,2003,1102,154.058824
1,2003,1103,168.705882


In [112]:
#Merge with the team name table to get the team name 
ratings3 = ratings2.merge(teams, on = 'TeamID')
ratings3.head(2)

,Season,TeamID,OrdinalRank,TeamName,FirstD1Season,LastD1Season
0,2003,1102,154.058824,Air Force,1985,2018
1,2004,1102,46.513514,Air Force,1985,2018


In [113]:
#Merge with the team name table to get the team name 

#testing - examples of how to query it
ratings3[ratings3['TeamName'] == 'Virginia'].sort_values('Season', ascending = False).head(3)


,Season,TeamID,OrdinalRank,TeamName,FirstD1Season,LastD1Season
4511,2017,1438,15.736842,Virginia,1985,2018
4510,2016,1438,4.720588,Virginia,1985,2018
4509,2015,1438,5.123077,Virginia,1985,2018


In [114]:
ratings3.Season.unique()
#only have ratings from 2003 - 2017

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017])

In [115]:
reg_season.head(2)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Diff,True_Outcome
0,1985,20,1228,81,1328,64,N,0,17,Y
1,1985,25,1106,77,1354,70,H,0,7,Y


### Modify schedule table 

In [116]:

#Add winning team rank, merging on team ID and Season
schedule_ratings = reg_season[reg_season['Season']>2002] \
                .merge(ratings3[['Season','TeamID','OrdinalRank','TeamName']], \
                left_on = ['WTeamID','Season'], right_on = ['TeamID','Season'])

schedule_ratings.rename(columns = {'OrdinalRank':'WTeamRank', 'TeamName':'WTeamName'}, inplace = True)

#Add losing team rank, merging on team ID and Season
schedule_ratings2 = schedule_ratings.merge(ratings3[['Season','TeamID','OrdinalRank','TeamName']], \
                left_on = ['LTeamID','Season'], right_on = ['TeamID','Season'])

schedule_ratings2.rename(columns = {'OrdinalRank':'LTeamRank', 'TeamName':'LTeamName'}, inplace = True)

#Drop duplicate column
schedule_ratings2.drop(['TeamID_x','TeamID_y'], 1, inplace = True)

#Example - all UVA wins from 2017 where the differential is greater than 5
schedule_ratings2[(schedule_ratings2['Season'] == 2017) & (schedule_ratings2['WTeamName'] == 'Virginia') \
    & (schedule_ratings2['True_Outcome'] == 'Y')] \
    .sort_values('DayNum').head(3)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Diff,True_Outcome,WTeamRank,WTeamName,LTeamRank,LTeamName
73516,2017,11,1438,76,1422,51,A,0,25,Y,15.736842,Virginia,109.819444,UNC Greensboro
71572,2017,15,1438,72,1383,32,H,0,40,Y,15.736842,Virginia,345.943662,St Francis NY
76507,2017,20,1438,62,1463,38,H,0,24,Y,15.736842,Virginia,137.652778,Yale


# Basic Naive Bayes 

In [117]:
""" Lets use only the both team's seed and the avg. rating to predict winner using Naive Bayes """
tourney_seeds.head(2)

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210


In [118]:
tourney_results.head(2)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0


In [119]:
#Step 1: Pull in the features (seeds and ratings)
tourney_results1 = tourney_results.merge(tourney_seeds, left_on = ['WTeamID','Season'] \
                                         , right_on = ['TeamID','Season'] )

tourney_results2 = tourney_results1.merge(tourney_seeds, left_on = ['LTeamID','Season'] \
                                         , right_on = ['TeamID','Season'] )

tourney_results3 = tourney_results2.merge(ratings3, left_on = ['WTeamID','Season'] \
                                         , right_on = ['TeamID','Season'] )

tourney_results4 = tourney_results3.merge(ratings3, left_on = ['LTeamID','Season'] \
                                         , right_on = ['TeamID','Season'] )

tourney_results5 = tourney_results4[['Season','WTeamID', 'LTeamID', 'TeamName_x', 'TeamName_y', \
                                    'Seed_x', 'Seed_y','OrdinalRank_x','OrdinalRank_y']] \
    .rename(columns = {'Seed_x':'WSeed','Seed_y': 'LSeed','TeamName_x' : 'WTeamName', \
                          'TeamName_y' : 'LTeamName', 'OrdinalRank_x' : 'WRank','OrdinalRank_y' : 'LRank'})

tourney_results5.head()

,Season,WTeamID,LTeamID,WTeamName,LTeamName,WSeed,LSeed,WRank,LRank
0,2003,1421,1411,UNC Asheville,TX Southern,X16b,X16a,242.470588,238.500000
1,2003,1112,1436,Arizona,Vermont,Z01,Z16,2.611111,153.176471
2,2003,1112,1211,Arizona,Gonzaga,Z01,Z09,2.611111,42.617647
3,2003,1112,1323,Arizona,Notre Dame,Z01,Z05,2.611111,21.611111
4,2003,1113,1272,Arizona St,Memphis,Z10,Z07,35.764706,23.555556


In [120]:
#selecting specific columns of tourney results
tourney_results6 = tourney_results5[['Season', 'WTeamID', 'LTeamID', 'WTeamName', 'LTeamName', \
                                    'WSeed', 'LSeed','WRank', 'LRank']]

#randomizing dataset
tourney_results6 = tourney_results6.sample(frac=1).reset_index(drop=True)

#splitting dataframe into two equal halves, so we can add multiple classifications for W/L
if (len(tourney_results6) % 2 > 0):
    print(len(tourney_results6))
    tourney_results6 = tourney_results6[:-1]
    print('trimming dataframe to even length so we can split')
    print(len(tourney_results6))
    
tourney_results6 = np.split(tourney_results6, 2)

981
trimming dataframe to even length so we can split
980


In [123]:
#first half of new df
#0 is winners
tourney_results6[0]['Winner'] = 1
#renaming columns
tourney_results6[0].rename(columns = 
                       {
                            'LRank': 'Team2Rank',
                           'LSeed': 'Team2Seed',
                           'LTeamID': 'Team2ID',
                           'LTeamName': 'Team2Name',
                           'WRank': 'Team1Rank',
                           'WSeed': 'Team1Seed',
                           'WTeamID': 'Team1ID',
                           'WTeamName': 'Team1Name',
                       }, inplace = True)


#second half of new df
#0 is losers
tourney_results6[1]['Winner'] = 2
#renaming columns
tourney_results6[1].rename(columns = 
                       {
                            'LRank': 'Team1Rank',
                           'LSeed': 'Team1Seed',
                           'LTeamID': 'Team1ID',
                           'LTeamName': 'Team1Name',
                           'WRank': 'Team2Rank',
                           'WSeed': 'Team2Seed',
                           'WTeamID': 'Team2ID',
                           'WTeamName': 'Team2Name',
                       }, inplace = True)

In [151]:
#unioning two dataframe together
tourney_results7 = pd.concat([tourney_results6[0], tourney_results6[1]])

In [152]:
tourney_results7.head(2)

,Season,Team1ID,Team1Name,Team1Rank,Team1Seed,Team2ID,Team2Name,Team2Rank,Team2Seed,Winner
0,2009,1345,Purdue,17.490196,Z05,1320,Northern Iowa,69.553191,Z12,1
1,2015,1326,Ohio St,23.461538,Z10,1433,VA Commonwealth,28.646154,Z07,1


In [176]:
#Format seeds and rank

def removeletters(x):
    return re.sub('[^0-9]','', x)

tourney_results7['Team1Seed'] = tourney_results7['Team1Seed'].apply(lambda x: removeletters(x)).apply(int)
tourney_results7['Team2Seed'] = tourney_results7['Team2Seed'].apply(lambda x: removeletters(x)).apply(int)


In [177]:
#testing removal
tourney_results7.Team2Seed.value_counts().head(3)

1    129
2    104
3     90
Name: Team2Seed, dtype: int64

In [180]:
features = ['Team1Rank', 'Team1Seed','Team2Rank','Team2Seed']

scaled_features = {}
for each in features:
    mean, std = tourney_results7[each].mean(), tourney_results7[each].std()
    scaled_features[each] = [mean, std]
    tourney_results7.loc[:, each] = (tourney_results7[each] - mean)/std


tourney_results7.head()

,Season,Team1ID,Team1Name,Team1Rank,Team1Seed,Team2ID,Team2Name,Team2Rank,Team2Seed,Winner
0,2009,1345,Purdue,-0.488565,-0.435683,1320,Northern Iowa,0.650119,1.093827,1
1,2015,1326,Ohio St,-0.361729,0.626958,1433,VA Commonwealth,-0.223001,0.027422,1
2,2010,1210,Georgia Tech,-0.091470,0.626958,1329,Oklahoma St,-0.043860,0.027422,1
3,2009,1173,Dayton,0.322188,0.839486,1452,West Virginia,-0.448559,-0.185859,1
4,2016,1292,MTSU,1.512582,1.689598,1277,Michigan St,-0.754699,-1.038983,1


In [187]:
tourney_results8 = tourney_results7[['Team1Rank','Team2Rank','Team1Seed','Team2Seed','Winner']]

In [195]:
features = tourney_results8.values[:,:(tourney_results8.shape[1] -2)] #features sans target var
target = tourney_results8.values[:,(tourney_results8.shape[1] -1)] #ensure that target var is last col
features_train, features_test, target_train, target_test = train_test_split(features, \
                                                target, test_size = 0.2, random_state = 69)
target_test

array([1., 2., 2., 1., 2., 1., 1., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1.,
       2., 1., 1., 2., 1., 1., 2., 1., 1., 1., 2., 2., 2., 2., 1., 2., 2.,
       1., 1., 2., 2., 1., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 1., 2.,
       2., 1., 2., 2., 2., 1., 1., 1., 1., 1., 2., 2., 1., 2., 2., 2., 1.,
       1., 1., 1., 2., 1., 2., 2., 1., 1., 2., 1., 1., 2., 1., 1., 1., 2.,
       1., 2., 1., 2., 2., 2., 1., 1., 1., 1., 1., 2., 1., 2., 2., 1., 1.,
       1., 2., 2., 2., 1., 1., 2., 2., 1., 2., 2., 2., 1., 2., 1., 2., 1.,
       2., 2., 1., 1., 1., 1., 2., 2., 2., 1., 2., 1., 1., 1., 2., 2., 2.,
       1., 1., 2., 1., 1., 2., 1., 2., 1., 1., 1., 2., 1., 2., 1., 1., 2.,
       2., 2., 1., 2., 1., 2., 2., 2., 1., 2., 1., 1., 2., 2., 1., 2., 2.,
       2., 2., 2., 2., 2., 1., 1., 2., 2., 1., 1., 2., 2., 2., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 1., 1., 1.])

In [196]:
clf = GaussianNB()
clf.fit(features_train, target_train)
target_pred = clf.predict(features_test)

accuracy_score(target_test, target_pred, normalize = True)


0.6785714285714286

In [197]:
# Import confusion matrix functionality
from sklearn.metrics import confusion_matrix as sk_confusion_matrix

# Create and format a confusion matrix
def conf_matrix(y_test, y_predict):

    # Create the raw confusion matrix
    conf = sk_confusion_matrix(y_test, y_predict)

    # Format the confusion matrix nicely
    conf = pd.DataFrame(data=conf)
    conf.columns.name = 'Predicted label'
    conf.index.name = 'Actual label'

    # Return the confusion matrix
    return conf

In [198]:
conf_matrix(target_test, target_pred)

Predicted label,0,1
Actual label,,
0,80,18
1,45,53


In [199]:
from sklearn import metrics
print(metrics.classification_report(target_test, target_pred))

             precision    recall  f1-score   support

        1.0       0.64      0.82      0.72        98
        2.0       0.75      0.54      0.63        98

avg / total       0.69      0.68      0.67       196

